Web Scraping

In [1]:
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import time

# Select season year
num_of_seasons = 2
season_begin_year = 2022
all_matches = []

for seasonNum in range(int(num_of_seasons)):
	season_begin_year = season_begin_year + seasonNum
	season_end_year = season_begin_year + 1

	standings_url = f"https://fbref.com/en/comps/9/{season_begin_year}-{season_end_year}/{season_begin_year}-{season_end_year}-Premier-League-Stats"

	season_year = standings_url.split("/")[-2]

	data = requests.get(standings_url)
	soup = BeautifulSoup(data.text)

	standings_table = soup.select('table.stats_table')[0]

	all_links = [l.get('href') for l in standings_table.find_all('a')]

	print(f'--Season {season_begin_year}-{season_end_year} Scraping...--')

	# Get information link for each team
	team_links = [l for l in all_links if '/squads/' in l]
	team_urls = [f'https://fbref.com{l}' for l in team_links]

	for team_url in team_urls:
		team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
		data = requests.get(team_url)

		matches = pd.read_html(StringIO(data.text), match="Scores & Fixtures")[0]

		soup = BeautifulSoup(data.text)
		links = soup.find_all('a')
		links = [l.get("href") for l in links]
		shooting_links = [l for l in links if l and 'all_comps/shooting/' in l]

		shooting_data = requests.get(f"https://fbref.com{shooting_links[0]}")
		shooting = pd.read_html(StringIO(shooting_data.text), match="Shooting")[0]
		shooting.columns = shooting.columns.droplevel()

		try:
			shooting_selected = shooting[[
				"Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]]
			team_data = matches.merge(shooting_selected, on="Date")
		except ValueError:
			continue

		team_epl_data = team_data[team_data["Comp"] == "Premier League"].copy()
		team_epl_data["Season"] = season_year
		team_epl_data["Team"] = team_name
		all_matches.append(team_epl_data)
		print(f'{len(all_matches)}: {team_name}')
		time.sleep(10)

	print(f'--Season {season_begin_year}-{season_end_year} scraping completed--')
	match_df = pd.concat(all_matches)

--Season 2022-2023 Scraping...--
1: Manchester City
2: Arsenal
3: Manchester United
4: Newcastle United
5: Liverpool
6: Brighton and Hove Albion
7: Aston Villa
8: Tottenham Hotspur
9: Brentford
10: Fulham
11: Crystal Palace
12: Chelsea
13: Wolverhampton Wanderers
14: West Ham United
15: Bournemouth
16: Nottingham Forest
17: Everton
18: Leicester City
19: Leeds United
20: Southampton
--Season 2022-2023 scraping completed--
--Season 2023-2024 Scraping...--
21: Manchester City
22: Arsenal
23: Liverpool
24: Aston Villa
25: Tottenham Hotspur
26: Chelsea
27: Newcastle United
28: Manchester United
29: West Ham United
30: Crystal Palace
31: Brighton and Hove Albion
32: Bournemouth
33: Fulham
34: Wolverhampton Wanderers
35: Everton
36: Brentford
37: Nottingham Forest
38: Luton Town
39: Burnley
40: Sheffield United
--Season 2023-2024 scraping completed--


In [2]:
# Check data information
match_df.info()
match_df

<class 'pandas.core.frame.DataFrame'>
Index: 1520 entries, 1 to 40
Data columns (total 28 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           1520 non-null   object 
 1   Time           1520 non-null   object 
 2   Comp           1520 non-null   object 
 3   Round          1520 non-null   object 
 4   Day            1520 non-null   object 
 5   Venue          1520 non-null   object 
 6   Result         1520 non-null   object 
 7   GF             1520 non-null   object 
 8   GA             1520 non-null   object 
 9   Opponent       1520 non-null   object 
 10  xG             1520 non-null   float64
 11  xGA            1520 non-null   float64
 12  Poss           1520 non-null   float64
 13  Attendance     1518 non-null   float64
 14  Captain        1520 non-null   object 
 15  Formation      1520 non-null   object 
 16  Opp Formation  1520 non-null   object 
 17  Referee        1520 non-null   object 
 18  Match Report   

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt,Season,Team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1,1,2022-2023,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0,0,2022-2023,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0,0,2022-2023,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0,0,2022-2023,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0,0,2022-2023,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,2024-04-24,20:00,Premier League,Matchweek 29,Wed,Away,L,2,4,Manchester Utd,...,Match Report,NaN,10.0,4.0,17.8,1.0,0,0,2023-2024,Sheffield United
37,2024-04-27,15:00,Premier League,Matchweek 35,Sat,Away,L,1,5,Newcastle Utd,...,Match Report,NaN,15.0,4.0,13.5,0.0,0,0,2023-2024,Sheffield United
38,2024-05-04,15:00,Premier League,Matchweek 36,Sat,Home,L,1,3,Nott'ham Forest,...,Match Report,NaN,16.0,4.0,18.0,0.0,1,1,2023-2024,Sheffield United
39,2024-05-11,15:00,Premier League,Matchweek 37,Sat,Away,L,0,1,Everton,...,Match Report,NaN,13.0,1.0,21.0,0.0,0,0,2023-2024,Sheffield United


In [3]:
# Export cleaned data set
match_df.to_csv(f'eql_matchData_22_24.csv', encoding="utf-8-sig")

file_path = 'eql_matchData_22_24.csv'

# Check if the file exists
if os.path.exists(file_path):
    print(f"File {file_path} was successfully exported.")
else:
    print(f"File {file_path} was NOT exported.")

File eql_matchData_22_24.csv was successfully exported.
